# generate_error_table

## imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import os

%matplotlib inline

## load data

In [ ]:
df0 = pd.read_csv('final_data/equaldistinctcount_20230221.csv')
df1 = pd.read_csv('final_data/equiheight_20230221.csv')
df2 = pd.read_csv('final_data/equiwidth_20230221.csv')
df3 = pd.read_csv('final_data/gdy_20230221.csv')
df4 = pd.read_csv('final_data/maxdiff_20230221.csv')
df5 = pd.read_csv('final_data/maxdiffarea_20230309.csv')
df = pd.concat([df0, df1, df2, df3, df4, df5], axis=0)

## generate errors

In [ ]:
def generate_errors(df):
    df = df.copy()  # Convenience
    
    df['absolute_error'] = np.absolute(df['estimated_output'] - df['real_output'])
    df['relative_error'] = df['absolute_error'] / df['real_output']
    df['x'] = df['estimated_output'] / df['real_output']
    df['1/x'] = 1 / df['x']
    df['q_error'] = df[['x','1/x']].max(axis=1)
    df['real_output_lower_bound'] = df['real_output']
    df['real_output_lower_bound'] = df['real_output_lower_bound'].clip(1)
    df['estimated_output_lower_bound'] = df['estimated_output']
    df['estimated_output_lower_bound'] = df['estimated_output_lower_bound'].clip(1)
    df['pseudo_x'] = df['estimated_output_lower_bound'] / df['real_output_lower_bound']
    df['pseudo_1/x'] = 1 / df['pseudo_x']
    df['pseudo_q_error'] = df[['pseudo_x','pseudo_1/x']].max(axis=1)
    df['mean_squared_error'] = df['absolute_error'] ** 2
    df.drop(['x', '1/x', 'real_output_lower_bound', 'estimated_output_lower_bound', 'pseudo_x', 'pseudo_1/x'], axis=1, inplace = True)

    return df

## result

### with_NULLs

In [ ]:
df_with_errors = generate_errors(df)
df_with_errors

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

### without_NULLs

In [ ]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

# print as the resulting table is cut by Jupyter
for operator in pd.unique(df_with_errors.operator_type):
    filtered = df_with_errors.query("operator_type == @operator")
    result = filtered.groupby(["operator_type",
                               "benchmark",
                               "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                                 mean_absolute_error=("absolute_error", np.mean),
                                                 mean_relative_error=("relative_error", np.mean),
                                                 mean_q_error=("q_error", np.mean),
                                                 mean_pseudo_q_error=("pseudo_q_error", np.mean))
    display(result)
    melted_results = pd.melt(result.reset_index(), id_vars=["benchmark", "histogram"],
                             value_vars=["root_mean_squared_error", "mean_absolute_error",
                                         "mean_relative_error", "mean_q_error", "mean_pseudo_q_error"],
                             var_name="metric")
    g = sns.FacetGrid(melted_results, col="benchmark", row="metric", sharey=False, aspect=2)
    g.map_dataframe(sns.barplot, x="histogram", y="value", hue="histogram", palette="Set3")
    g.add_legend()
    plt.savefig(f"{operator}_facet.pdf")
    plt.show()
    display(g)


## compare_operators

In [ ]:
grouped = df.groupby(["benchmark", "histogram", "query", "operator_type"]).size()

queries = df[["benchmark", "query"]].drop_duplicates()

# super inefficient, doesn't matter for now
op_counts = {}
for _, outer_benchmark, outer_query in queries.itertuples():
    for (_1, benchmark, histogram, query, operator_type, count) in grouped.reset_index().itertuples():
        if outer_benchmark == benchmark and outer_query == query:
            if (benchmark, query, operator_type) not in op_counts:
                op_counts[(benchmark, query, operator_type)] = count
                continue

            if op_counts[(benchmark, query, operator_type)] != count:
                print(f"WARNING: different value of {count} (previously {op_counts[(benchmark, query, operator_type)]}) for {benchmark}, query {query}, op {operator_type}")
